In [1]:
# declaraciones de librerías
using DataStructures

In [2]:
type gFlows
    nVertices::Int64
    capacidades::Array{Int64,2} #arco[i,j] indica la distancia entre i y j 
end

type arcoFlow
    origen::Int64
    destino::Int64
    capacidad::Int64
end

type gFlowsSparse
    nVertices::Int64
    cte::Int64
    nArcos::Int64
    arcos::Array{arcoFlow,1} #arco[i,j] indica la distancia entre i y j 
    pOrigen::Array{Int64,1}
    destinos::Array{Int64,1}
    pDestino::Array{Int64,1}
    origenes::Array{Int64,1}
    capacidad::Dict{Int64,Int64}
end


In [3]:
function generarGrafoDense(n::Int64,maxValue::Int64)
    a=Array{Int64}(n,n)
    rand!(a,1:maxValue)
    for i in 1:n
        a[i,i]=0
    end
    for i in 1:n
        for j in i+1:n
            if i!=j
                if randn()<0.0
                    a[i,j]=0.0
                else
                    a[j,i]=0.0
                end
            end
        end
    end
    return gFlows(n,a) 
end

generarGrafoDense (generic function with 1 method)

# Versión de Edmons y Karp del algoritmo de Ford-Fulkerson

El algoritmo utiliza la idea de red residual. La red residual es aquella por la que puede pasarse flujo (ya sea avanzando flujo por la red o reduciendo flujo por la misma). En la red residual busca un camino (en el caso de Edmons-Karp, un camino con el número mínimo de arcos) y aumenta la cantidad transportada por los arcos del camino hasta saturarlo. Este camino se encuentra mediante un algoritmo tipo Breadth-first

El proceso se repite hasta que no puede pasarse flujo.


In [11]:
function conectividadBreadthFirst(G::gFlows,flows::Array{Int64,2},origen::Int64,destino::Int64)
    ∞=1000000
    #arrays para construir evaluación
    color=falses(G.nVertices)
    π=Array(Int64,G.nVertices)
    fill!(π,-1)
    #inicializamos para el vertice
    color[origen]=true
    Q = Deque{Int64}()
    push!(Q,origen)
    while isempty(Q)==false
        u=shift!(Q)
        #println("\t",u)
        for i in 1:G.nVertices
            if color[i]==false && flows[u,i]>0
                if i==destino
                    π[i]=u
                    return π
                end
                color[i]=true
                π[i]=u
                push!(Q,i)
            end
        end
    end
    return π
end


function ffDense(g::gFlows,origin::Int64,destination::Int64)
    #flows=Array{Int64}(g.nVertices,g.nVertices)
    ∞=1000000
    flow=deepcopy(g.capacidades)
    #println("flow: ",flow)
    maxFlow=0
    π=conectividadBreadthFirst(g,flow,origin,destination)
    while π[destination]!= -1
        #print("pi: ",π)
        extraFlow=∞
        c=destination
        while π[c] != -1
            extraFlow=min(extraFlow,flow[π[c],c])
            c=π[c]
        end
        c=destination
        while π[c] != -1
            flow[π[c],c] -= extraFlow
            flow[c,π[c]] += extraFlow
            c=π[c]
        end
        maxFlow += extraFlow
        #println("\t",extraFlow,"\t",maxFlow)
        π=conectividadBreadthFirst(g,flow,origin,destination)
    end
    return maxFlow,flow
end

ffDense (generic function with 1 method)

In [12]:
#g=generarGrafoDense(10,100)
maxFlow,flows=ffDense(g,1,10)
println("flujo: ",maxFlow)

flujo: 104


# Algoritmo de Dinic

# Algoritmo de Goldberg y Tarjan (Push-relabel)